In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1" 

In [3]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc
import wandb 
from fastxtend.vision.all import EMACallback


In [4]:
!ls ../data/

Ribonanza_bpp_files	  sequence_libraries
all_test_data.npy	  split
eterna_openknot_metadata  supplementary_silico_predictions
rcentral.parquet	  test_sequences.csv
rmdb_data.v1.3.0.csv	  test_sequences.parquet
rna-files.zip		  train_corrected.parquet
rna_files		  train_data.csv
sample_submission.csv	  train_data.parquet


In [5]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    pathss = Path("../eda/train_ss_vienna_rna.parquet")
    split_id = Path('../eda/fold_split.csv')
    pl_data = Path('../data/rna_files/final_comb_PLfoldsEXft_tot.pickle')
    bs = 32
    num_workers = 4
    device = 'cuda'
    seed = 2023
    out = 'exp_32_v3_psd_v1'
    dataset_name = 'RNA_DatasetBaselineSplitssbppV6SAVEDwithFMPSD'
    dataset_val = 'RNA_DatasetBaselineSplitssbppV6SAVEDwithFM'
    sn_train = False
    
    model_name = 'RNA_ModelV25'
    model_kwargs = dict(dim=192 * 2,
        depth=4,
        head_size=32,
        drop_pat_dropout=0.2,
        dropout=0.2,
        bpp_transfomer_depth = 6)

   

    epoch = 5
    lr = 5e-5
    wd = 0.05
    pct_start = 0.01
    
    
    md_wt = 'exp_32_v3_psd/models/model.pth'
    
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

def class_to_dict(cls):
    # Create a dictionary from the class attributes
    return {key: value for key, value in cls.__dict__.items() if not key.startswith("__") and not callable(value)}

In [6]:



with open(str(CFG.pl_data), "rb") as f: 
    data = pickle.load(f)
orig_test_csv = CFG.path/'test_sequences.parquet'
train_data = CFG.path/'train_data.parquet'
folds_split = CFG.split_id


ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(data, orig_test_csv, train_data, folds_split, mode='train')

dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_size=CFG.bs,
            drop_last=True, 
            shuffle =True,
            num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)




split = pd.read_csv(CFG.split_id)
df = pd.read_parquet(CFG.path/'train_corrected.parquet')
df = pd.merge(df, split, on='sequence_id')
#df = df.query("SN_filter==1").reset_index(drop=True)
df_train = df.query('is_train==True').reset_index(drop=True)
df_valid = df.query('is_train==False').reset_index(drop=True)

ds_val = getattr(rnacomp.dataset, CFG.dataset_val)(df_valid, mode='eval')
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val,  
                                                     batch_size=CFG.bs, 
                                                       drop_last=False,
                                                     num_workers=CFG.num_workers),
                         CFG.device)


data = DataLoaders(dl_train,dl_val)
del split
del df
gc.collect()




7243

In [ ]:
#the training was interepeted on epoch 3 
#so i had to restart from scratch but loading weights from epoch 3
learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)(**CFG.model_kwargs).cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                    #WandbCallback(log_preds=False),
                    CSVLogger(),
                     EMACallback(replace_weights=True),
                    SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],
                metrics=[MAE()]).to_fp16() 
learn.model.load_state_dict(torch.load(CFG.md_wt), strict=False)
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)




epoch,train_loss,valid_loss,mae,time


In [ ]:
res

In [ ]:
res